# Homework 1, Part 2 (N-gram Language Modeling)
Instructor: [Ziyu Yao at George Mason University](https://ziyuyao.org/)

Class: CS678 Fall 2025

**Total points: 80 points**

## Overview
This assignment includes three parts:
- Part 2.1: Constructing a bigram language model (40 points)
- Part 2.2: Evaluating the bigram language model (20 points)
- Part 2.3: Sampling sentences from the bigram language model (20 points)

You can run this notebook locally on your laptop or computer. Otherwise, running it on Google Colab (https://colab.research.google.com/) is also feasible -- if you go with this option, select "Upload" and then this notebook to upload it to Colab. Note that no matter how you will run this notebook, **do not modify existing code/markdown cells unless instructed.**

This assignment also comes with a LaTex project. To collect points from this assignment, you must revise the LaTex project file and fill out the answer blanks as instructed.

## Submission Guideline
When you complete the assignment,
1. Make sure to save your edits and **all outputs from executing the code cells (IMPORTANT)**. Submit this .ipynb file to Canvas.
2. Compile the PDF from the LaTex project. Submit the PDF to Gradescope.


## Google Drive Setup/Tentative Data Loading
Our assignment will be using a small sample of the Gutenberg corpus (https://www.gutenberg.org/) provided by the NLTK Python library. To make it easy, we have included the txt data ("nltk_gutenberg_austen_emma.txt") along with this assignment.

**If you are using Google Colab**, there are a few more steps for setting up the data access. **If you are running this notebook locally**, skip commands in this part and directly start from "Data Loading".

For Colab users, there are two ways how you can let your notebook access the provided txt file; you only need to choose one of them.

**(1) Mounting your Google Drive.** If you take this option, execute the following code and follow the pop-up instructions to give Colab the access. Please make sure to tick the option of allowing Colab to read/write files from/to your Google Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Your Google Drive is now mounted. Go to your Drive folder through https://drive.google.com and create a folder for CS678. Then upload "nltk_gutenberg_austen_emma.txt" to this folder. Now your Colab should be able to access this txt file from "/content/drive/My Drive/CS678/nltk_gutenberg_austen_emma.txt".

**(2) Uploading tentative data copy to Colab.** Alternatively, especially if you have concerns about giving Colab access to your Drive, run the following command to allow for tentative data upload. You will see a "Choose File" button. Click the button and select "nltk_gutenberg_austen_emma.txt" from your local folder.

In [25]:
from google.colab import files
files.upload()

Saving nltk_gutenberg_austen_emma.txt to nltk_gutenberg_austen_emma.txt


{'nltk_gutenberg_austen_emma.txt': b'A crowd in a little room -- Miss Woodhouse , you have the art of giving pictures in a few words .\nMr . Knightley \' s eyes had preceded Miss Bates \' s in a glance at Jane .\n" Well -- if you please ," said Mrs . Weston rather hesitating , " if you think she will be of any use ."\n" When I talked of your being altered by time , by the progress of years ," said John Knightley , " I meant to imply the change of situation which time usually brings .\n" He was four - and - twenty the 8th of last June , and my birthday is the 23rd just a fortnight and a day \' s difference -- which is very odd ."\nA man must be very much in love , indeed , to describe her so .\nEmma could not forgive her .\nYes , I see what she means , ( turning to Mr . Knightley ,) and I will try to hold my tongue .\nHe was wishing to get the better of his attachment to herself , she just recovering from her mania for Mr . Elton .\n" I thank you ; but I assure you you are quite mistake

## Data Loading

With the data access set up well, we now start to load in and process the txt file. The following code will read in the corpus and split it into the training and the test set.

In [26]:
# TODO: if necessary, modify the data path based on where you save it
DATA_PATH = "./nltk_gutenberg_austen_emma.txt"

# load data
sents_all = []
with open(DATA_PATH, "r") as f:
    for line in f.readlines():
        sents_all.append(line.strip().split())

total_num = len(sents_all)

sents_train = sents_all[:int(total_num*0.8)]
sents_test = sents_all[len(sents_train):]
print("Number of sentences in the training and the test corpus: %d and %d, respectively" % (
    len(sents_train), len(sents_test)))

Number of sentences in the training and the test corpus: 6173 and 1544, respectively


In this assignment, you will be instructed to build a bigram language model (LM) based on the Gutenberg training set, and then evaluate it on the test set.

**Before You Start:** A hint on debugging your LM is to use toy data which you can manually calculate the LM probabilities and verify the result. Example toy `sents_train` and `sents_test` corpora are shown in the following code block. Uncomment and execute the code block when you want to debug your model.

In [4]:
#alternatively, load the toy data
# sents_train = [
#    ["A", "B", "B", "C"],
#    ["A", "D", "C"],
#    ["E", "A"]
# ]
# sents_test = sents_train

In [27]:
from typing import List, Any
from math import isclose

In [28]:
print("The first 2 sentences from `sents_train`: ")
for idx in range(2):
    print(sents_train[idx]) # each sentence is a list of words

The first 2 sentences from `sents_train`: 
['A', 'crowd', 'in', 'a', 'little', 'room', '--', 'Miss', 'Woodhouse', ',', 'you', 'have', 'the', 'art', 'of', 'giving', 'pictures', 'in', 'a', 'few', 'words', '.']
['Mr', '.', 'Knightley', "'", 's', 'eyes', 'had', 'preceded', 'Miss', 'Bates', "'", 's', 'in', 'a', 'glance', 'at', 'Jane', '.']


### **NOTE: In this assignment, each sentence has been tokenized, and NO MORE preprocessing is required.**

## Part 2.1: Language Model Construction (40 points)

**Overview:** In this part, you will construct a bigram language model (LM) based on the `sents_train` corpus. A bigram LM models the probability of the next word given the current word, i.e., $p(w_t | w_{t-1})$. The construction of a bigram LM could be achieved by counting up the word or word-pair frequency:
$$p(w_t|w_{t-1}) = \frac{count(w_{t-1}, w_t)}{count(w_{t-1})}.$$

To deal with zero counts, *add-one smoothing* is commonly used:
$$p(w_t|w_{t-1}) = \frac{count(w_{t-1}, w_t) + 1}{count(w_{t-1}) + |V|},$$
where $|V|$ is the size of the vocabulary.
**Your implementation of the LM should be based on this add-one smoothing formulation.**

---
**Step 1 -- Construct a vocabulary (15 points):** To get started, you will first need to construct a vocabulary based on the `sents_train` corpus. Also, don't forget the special start-of-sentence (`<s>`) and end-of-sentence (`</s>`) tokens -- your LM should eventually be able to model `p(w|<s>)` (i.e., how to start a sentence) and `p(</s>|w)` (i.e., when to stop a sentence).

First, create a counter for word types in the `sents_train` corpus **(5 points)**.

<font color='blue'>PLEASE INPUT YOUR ANSWER BELOW</font>

In [29]:
# TODO: create a counter of word type based on sents_train
from collections import Counter

word_counts = Counter()
for sentence in sents_train:
    # Add start and end tokens before counting
    processed_sentence = ["<s>"] + sentence + ["</s>"]
    word_counts.update(processed_sentence)


# Print the total number of unique words found
print("Total number of unique words (including duplicates):", sum(word_counts.values()))
print("Number of unique word types:", len(word_counts))

Total number of unique words (including duplicates): 165444
Number of unique word types: 7113


Then, create a vocabulary based on your word type counter **(5 points)**.

As we explained in class, it would be difficult for a count-based n-gram LM to learn good distributions for infrequent words. Therefore, we typically only keep frequent words in the vocabulary and replace others (i.e., infrequent words) with a special token `UNK`. In this assignment, we will keep only words appearing *at least* twice in `sents_train` in the vocabulary, and any words appearing fewer than two times will be replaced by `UNK`.

As a reminder, your vocabulary should also include `<s>` and `</s>`, which are necessary for modeling the start and stop of a sentence.

<font color='blue'>PLEASE INPUT YOUR ANSWER BELOW</font>

In [30]:
# TODO: create a vocabulary with UNK
vocabulary = set()
for word, count in word_counts.items():
    if count >= 2:
        vocabulary.add(word)

# UNK is always included
vocabulary.add("UNK")

# Convert the set to a list for consistent ordering (optional, but good practice)
vocabulary = list(vocabulary)

Now, show the size of your vocabulary (i.e., how many distinct words in your vocab, including UNK) **(2 points)**:

<font color='blue'>PLEASE INPUT YOUR ANSWER BELOW</font>

In [31]:
# TODO: show the size of your vocabulary
vocab_size =  len(vocabulary) # replace it with your implementation

print("Vocabulary size (including UNK):", vocab_size)

Vocabulary size (including UNK): 4183


Now, let's use this vocabulary to index sentences in `sents_train`, what are the most frequent words (including UNK)? Show the top 10 word types with their counts. **(3 points)**

<font color='blue'>PLEASE INPUT YOUR ANSWER BELOW</font>

In [32]:
# TODO: Show the top 10 word types and their counts, one in a row, i.e.,
# word1, count1
# word2, count2
# ...
# word10, count10

# Create a counter for all tokens, including <s> and </s>, after processing sentences
all_token_counts = Counter()
for sentence in sents_train:
    # Add start and end tokens and map words to vocabulary or UNK
    processed_sentence = ["<s>"] + [word if word in vocabulary else "UNK" for word in sentence] + ["</s>"]
    all_token_counts.update(processed_sentence)


# Get the top 10 most frequent tokens and their counts
top_10_tokens = all_token_counts.most_common(10)

# Print the top 10 tokens and their counts in the requested format
for token, count in top_10_tokens:
    print(f"{token} {count}")

, 9129
<s> 6173
</s> 6173
. 5513
to 4115
the 3805
and 3748
of 3386
UNK 2931
I 2518


For sanity check, with the toy dataset, the vocabulary size is
```
6
```
and the token counts are:
```
<s> 3
A 3
</s> 3
B 2
C 2
UNK 2
```

---
**Step 2 -- Build the bigram LM (25 points):** Next, implement the bigram LM $p(w_t|w_{t-1})$ (with add-one smoothing) based on the `sents_train` corpus.

First, accumulate the bigrams in `sents_train` and create a bigram counter **(10 points)**:

<font color='blue'>PLEASE INPUT YOUR ANSWER BELOW</font>

In [33]:
# TODO: Please input your code below:
bigram_counts = Counter()
for sentence in sents_train:
    # Add start and end tokens to the sentence
    processed_sentence = ["<s>"] + [word if word in vocabulary else "UNK" for word in sentence] + ["</s>"]
    # Update bigram counts
    for i in range(len(processed_sentence) - 1):
        bigram_counts[(processed_sentence[i], processed_sentence[i+1])] += 1

# Print some sample bigram counts for verification
print("Sample bigram counts:", list(bigram_counts.items())[:10])

Sample bigram counts: [(('<s>', 'A'), 69), (('A', 'crowd'), 1), (('crowd', 'in'), 2), (('in', 'a'), 145), (('a', 'little'), 117), (('little', 'room'), 4), (('room', '--'), 5), (('--', 'Miss'), 5), (('Miss', 'Woodhouse'), 138), (('Woodhouse', ','), 92)]


For sanity check, the bigram counter includes:
```
{('<s>', 'A'): 2, ('C', '</s>'): 2, ('A', 'B'): 1, ('B', 'B'): 1, ('B', 'C'): 1, ('A', 'UNK'): 1, ('UNK', 'C'): 1, ('<s>', 'UNK'): 1, ('UNK', 'A'): 1, ('A', '</s>'): 1}
```

For how many times in `sent_train` does a sentence start with the word "I"?

<font color='blue'>PLEASE INPUT YOUR ANSWER BELOW</font>

In [34]:
# TODO: Complete the code below to show the number of occurrences of "I" being the first word in sent_train
count_starts_with_I = 0
for sentence in sents_train:
    if sentence and sentence[0] == "I":
        count_starts_with_I += 1

print("Sentences in sent_train starts with the word `I` for times:", count_starts_with_I)

Sentences in sent_train starts with the word `I` for times: 502


Next, create the bigram language model (with add-one smoothing) by completing the following code block. **(10 points)**

Note 1: The last few lines of code (now commented) are checking $\sum_{w_t \in \text{Vocab}} P(w_t | w_{t-1}) = 1$, i.e., whether the probability mass of `P(w_t|w_{t-1})` sums to 1 when enumerating **all possible next word `w_t` in the vocabulary**. However, the code lines only work when the LM is defined as a dictionary `model_p` which stores `(w_{t-1}, w_t)` as key and its probability as value. You can write your own check if you are not following this data structure.

Note 2: There is no need to learn $P(w_t | </s>)$ because a sentence will never start with `</s>`. As shown in the probability mass check, we do not check the validity of $P(w_t | </s>)$.

Note 3: You do not need special handling to prevent `<s>` to be generated (i.e., non-zero $P(</s>|w_{t-1})$). In practice, when the training corpus is large, a properly learned LM will not assign `<s>` with a high probability in the middle of the sentence generation.

<font color='blue'>PLEASE INPUT YOUR ANSWER BELOW</font>

In [35]:
import time

start_time = time.time()

# TODO: create your bigram LM
model_p = dict() # a dict of {(w_t-1, w_t): probability}

# Calculate unigram counts including <s> and </s> for denominators in add-one smoothing
unigram_counts = Counter()
for sentence in sents_train:
    processed_sentence = ["<s>"] + [word if word in vocabulary else "UNK" for word in sentence] + ["</s>"]
    unigram_counts.update(processed_sentence)


vocab_size = len(vocabulary)

for (w_tm1, w_t), bigram_count in bigram_counts.items():
    # The denominator is count(w_t-1) + |V|
    # We use the unigram count of w_t-1 as the count(w_t-1)
    denominator = unigram_counts[w_tm1] + vocab_size
    model_p[(w_tm1, w_t)] = (bigram_count + 1) / denominator

# We can iterate through all possible (w_tm1, w_t) pairs where w_tm1 has a count > 0
for w_tm1 in unigram_counts:
    if w_tm1 == "</s>": # No bigrams start with </s>
        continue
    for w_t in vocabulary:
        if (w_tm1, w_t) not in model_p:
            # This bigram was not seen in the training data, so its count is 0
            bigram_count = 0
            denominator = unigram_counts[w_tm1] + vocab_size
            model_p[(w_tm1, w_t)] = (bigram_count + 1) / denominator

# Print the bigram probabilities
for key, value in model_p.items():
    print(f"{key} {value}")

# Do not modify code after this line
end_time = time.time()
print("Spent %s for the bigram LM construction." % time.strftime("%Hh%Mm%Ss", time.gmtime(end_time-start_time)))




# The following lines verify the validity of the probability distribution -- it takes a while, please wait.
# If you receive an assertion error, then your implementation could be problematic.
# for w_tm1 in vocabulary:
#    if w_tm1 == "</s>": # not needed
#        continue
#    pr_mass = 0 # sum over different w's of p(w|w_tm1)
#    for w_t in vocabulary:
#        pr_mass += model_p[(w_tm1, w_t)]
#    assert isclose(pr_mass, 1.0), "Probability mass of %s should sum to 1" % w_tm1 # sum should equals to 1

Streaming output truncated to the last 5000 lines.
('visibly', 'coloured') 0.00023894862604540023
('visibly', 'connexions') 0.00023894862604540023
('visibly', 'sentences') 0.00023894862604540023
('visibly', 'live') 0.00023894862604540023
('visibly', 'wittier') 0.00023894862604540023
('visibly', 'gardens') 0.00023894862604540023
('visibly', 'risk') 0.00023894862604540023
('visibly', 'Beyond') 0.00023894862604540023
('visibly', 'confessed') 0.00023894862604540023
('visibly', 'heartily') 0.00023894862604540023
('visibly', 'leave') 0.00023894862604540023
('visibly', 'arm') 0.00023894862604540023
('visibly', 'shower') 0.00023894862604540023
('visibly', 'shaken') 0.00023894862604540023
('visibly', 'prosperity') 0.00023894862604540023
('visibly', 'premature') 0.00023894862604540023
('visibly', 'housekeeper') 0.00023894862604540023
('visibly', 'church') 0.00023894862604540023
('visibly', 'all') 0.00023894862604540023
('visibly', 'biscuits') 0.00023894862604540023
('visibly', 'lament') 0.000238

For sanity check, your bigram LM should give the following probabilities:
```
('<s>', '<s>') 0.1111111111111111
('<s>', 'A') 0.3333333333333333
('<s>', '</s>') 0.1111111111111111
('<s>', 'B') 0.1111111111111111
('<s>', 'C') 0.1111111111111111
('<s>', 'UNK') 0.2222222222222222
('A', '<s>') 0.1111111111111111
('A', 'A') 0.1111111111111111
('A', '</s>') 0.2222222222222222
('A', 'B') 0.2222222222222222
('A', 'C') 0.1111111111111111
('A', 'UNK') 0.2222222222222222
('B', '<s>') 0.125
('B', 'A') 0.125
('B', '</s>') 0.125
('B', 'B') 0.25
('B', 'C') 0.25
('B', 'UNK') 0.125
('C', '<s>') 0.125
('C', 'A') 0.125
('C', '</s>') 0.375
('C', 'B') 0.125
('C', 'C') 0.125
('C', 'UNK') 0.125
('UNK', '<s>') 0.125
('UNK', 'A') 0.25
('UNK', '</s>') 0.125
('UNK', 'B') 0.125
('UNK', 'C') 0.25
('UNK', 'UNK') 0.125
```

Can you show the most frequent 10 starting words (i.e., words following `<s>`) with their probabilities to three decimal places (e.g., 0.123)? **(5 points)**

<font color='blue'>PLEASE INPUT YOUR ANSWER BELOW</font>

In [36]:
# TODO: Show the most frequent 10 starting words and their probabilities, one in a row, e.g.,
# word1, prob1
# word2, prob2
# ...
# word10, prob10

# Get probabilities for words following <s>
start_word_probabilities = {}
for (w_tm1, w_t), prob in model_p.items():
    if w_tm1 == "<s>":
        start_word_probabilities[w_t] = prob

# Sort by probability in descending order and get the top 10
top_10_start_words = sorted(start_word_probabilities.items(), key=lambda item: item[1], reverse=True)[:10]

# Print the top 10 starting words and their probabilities
for word, prob in top_10_start_words:
    print(f"{word}, {prob:.3f}")

", 0.123
I, 0.049
She, 0.036
He, 0.028
It, 0.023
The, 0.022
Emma, 0.017
But, 0.015
Mr, 0.014
You, 0.013


## Part 2.2: Evaluation of Language Model (20 points)

**Overview:** In this part, you will implement the perplexity metric ($ppl$) and evaluate your bigram LM on the `sents_test` corpus ($D_{test}$). The math formulation of the perplexity is:
\begin{align}
    H(D_{test}) &= \frac{1}{\sum_{s \in D_{test}} |s|} \sum_{s \in D_{test}} -\log_2 P(s),\\
    ppl(D_{test}) &= 2^{H(D_{test})},
\end{align}
where $s \in D_{test}$ denotes a sentence in the test corpus, $|s|$ is its size (i.e., number of word tokens in the sentence `s`, *excluding* the extra `<s>` and `</s>`), $P(s)$ calculates the probability of a sentence $s$, and $H(D_{test})$ is the per-word cross entropy of the LM on the test corpus.

Since this part involves log calculation, you could use Python libraries such as `numpy` and `math`. **If you are running the notebook on Google Colab,** the libraries have been installed; **If you are running the notebook locally**, you may need to uncomment the following command and install the two libraries.

In [ ]:
# !pip3 install numpy # uncomment and run it if you haven't installed numpy

In [37]:
import numpy as np
import math

---
**Step 1 -- Calculate the log2 probability of a test sentence (10 points):** You will first implement the calculation of $\log_2 P(s)$ using the LM you have constructed in Part 2.1:
$$ \log_2 P(s) = \log_2 p(w_1| \text{<s>}) + \log_2 p(\text{</s>}|w_T) + \sum_{t=2}^T \log_2 p(w_t|w_{t-1}).$$

**Hint:** Don't forget to handle UNK tokens!

<font color='blue'>PLEASE INPUT YOUR ANSWER BELOW</font>

In [38]:
# TODO: The log2_P function should return a floating number indicating log2 P(s)
def log2_P(sent: List[str]) -> float:
    log_prob = 0.0
    # Add start and end tokens and map words to vocabulary or UNK
    processed_sentence = ["<s>"] + [word if word in vocabulary else "UNK" for word in sent] + ["</s>"]

    # Calculate log probability of the sentence
    for i in range(len(processed_sentence) - 1):
        bigram = (processed_sentence[i], processed_sentence[i+1])
        prob = model_p.get(bigram, 1 / (unigram_counts.get(processed_sentence[i], 0) + vocab_size))

        # Add the log2 probability to the total
        if prob > 0: # Avoid log(0)
            log_prob += math.log2(prob)
        else:
            pass

    return log_prob

Here's an example of how your $\log_2 P$ function will be used. The example shows the function output for the first sentence in the test set `sents_test`.

In [39]:
s = sents_test[0]
print("sentence:", s)
log2_pr = log2_P(s)
print("log2_P returns:", log2_pr)

sentence: ['The', 'extent', 'of', 'your', 'admiration', 'may', 'take', 'you', 'by', 'surprize', 'some', 'day', 'or', 'other', '."']
log2_P returns: -155.8266713670138


For sanity check, the return for the first sentence in the toy dataset is:
```
sentence: ['A', 'B', 'B', 'C']
log2_P returns: -9.169925001442312
```

---
**Step 2 -- Calculate the per-word cross entropy (10 point):** You will then implement $H(D_{test})$ by reusing the $log2\_P$ function defined in Step 1.

<font color='blue'>PLEASE INPUT YOUR ANSWER BELOW</font>

In [40]:
# TODO: The H function should return a floating number indicating H(sents)
def H(sents: List[List[str]]) -> float:
    total_log_prob = 0.0
    total_words = 0

    for sent in sents:
        # Calculate log2 P(s) for the sentence
        total_log_prob += log2_P(sent)

        # Add the number of words in the sentence
        total_words += len(sent)

    # Calculate the per-word cross entropy
    if total_words > 0:
        h_value = -total_log_prob / total_words
    else:
        h_value = 0.0 # Or handle as an error, depending on expected input

    return h_value

Here's an example how your $H$ function will be used:

In [41]:
sents_sample = [sents_test[0]]
print("sents:", sents_sample)
h_value = H(sents_sample)
print("H returns:", h_value)

sents: [['The', 'extent', 'of', 'your', 'admiration', 'may', 'take', 'you', 'by', 'surprize', 'some', 'day', 'or', 'other', '."']]
H returns: 10.38844475780092


For sanity check, the return for the first sentence in the toy dataset is:
```
sents: [['A', 'B', 'B', 'C']]
H returns: 2.292481250360578
```

---
**Step 3 -- Implement the perplexity metric:** After defining $H(D_{test})$, you can define the perplexity function $ppl$.

In [42]:
# The ppl function should return a floating number indicating the perplexity of the LM on `sents`
def ppl(sents: List[List[str]]) -> float:
    h_value = H(sents)
    return 2**h_value

**Finally,** let's evaluate the bigram LM you have constructed in Part 1 on the `sents_test` test corpus:

In [43]:
ppl_value = ppl(sents_test)
print("Perplexity:", ppl_value)

Perplexity: 527.0427483740344


For sanity check, the return for the toy dataset is:
```
Perplexity: 5.73568847053785
```

## Part 2.3: Sampling from Language Model (20 points)

**Overview:** The last part is about sampling different sentences from the learned language model. This means to repeatedly sample the next word based on the current word, until producing an end-of-sentence (`</s>`) token or the sentence length reaches a pre-defined limit (let's set `max_len`=50). Remember that each sentence should start from the special start-of-sentence (`<s>`) token.

In [44]:
max_len = 50

---
**Step 1 -- Greedy decoding (10 point):** Please sample one sentence from the bigram language model using greedy decoding, i.e., always choosing the most probable word as the next word, until producing an end-of-sentence token (`</s>`) or the sentence length reaches a pre-defined limit `max_len`.

<font color='blue'>PLEASE INPUT YOUR ANSWER BELOW</font>

In [45]:
# greedy decoding
sent = [] # used to store the generated words in your sentence

# TODO: Please input your code below:
current_word = "<s>"
while current_word != "</s>" and len(sent) < max_len:
    # Find the next word with the highest probability given the current word
    next_word = None
    max_prob = -1.0

    for (w_tm1, w_t), prob in model_p.items():
        if w_tm1 == current_word:
            if prob > max_prob:
                max_prob = prob
                next_word = w_t

    if next_word is not None:
        if next_word != "</s>":
            sent.append(next_word)
        current_word = next_word
    else:
        break


# print the decoded sentence
print(sent)

['"', 'I', 'am', 'sure', ',', 'and', 'the', 'UNK', ',', 'and', 'the', 'UNK', ',', 'and', 'the', 'UNK', ',', 'and', 'the', 'UNK', ',', 'and', 'the', 'UNK', ',', 'and', 'the', 'UNK', ',', 'and', 'the', 'UNK', ',', 'and', 'the', 'UNK', ',', 'and', 'the', 'UNK', ',', 'and', 'the', 'UNK', ',', 'and', 'the', 'UNK', ',', 'and']


---
**Step 2 -- Sampling by distribution (10 point):** To encourage diversity while maintaining a certain degree of "naturalness", we could sample the next word following the LM's next-word probability distribution, i.e., $w \sim p(w|w_{t-1})$. Please implement this sampling strategy. Same as before, the generation should end when producing an end-of-sentence token (`</s>`) or the sentence length reaches a pre-defined limit `max_len`.

**Hint:** You could use `numpy.random.choice` to implement the sampling.

<font color='blue'>PLEASE INPUT YOUR ANSWER BELOW</font>

In [46]:
np.random.seed(1234) # for grading purpose, do not change the random seed

sent = [] # used to store your sentence

# TODO: Please input your code below:
current_word = "<s>"
while current_word != "</s>" and len(sent) < max_len:
    # Get possible next words and their probabilities given the current word
    next_word_probs = {w_t: prob for (w_tm1, w_t), prob in model_p.items() if w_tm1 == current_word}

    # Extract words and their probabilities as lists for numpy.random.choice
    words = list(next_word_probs.keys())
    probabilities = list(next_word_probs.values())

    # Sample the next word based on the distribution
    if words:
        next_word = np.random.choice(words, p=probabilities)

        if next_word != "</s>": # Don't add </s> to the sentence list
            sent.append(next_word)
        current_word = next_word
    else:
        break

# print the decoded sentence
print(sent)

[np.str_('She'), np.str_('keep'), np.str_('reflected'), np.str_('liberal'), np.str_('Lord'), np.str_('dwelt'), np.str_('gladly'), np.str_('_There_'), np.str_('exert'), np.str_('disappointed'), np.str_('entering'), np.str_('basin'), np.str_('fresh'), np.str_('lips'), np.str_('governess'), np.str_('oblige'), np.str_('form'), np.str_('acquaintance'), np.str_('sensation'), np.str_('alluded'), np.str_('examine'), np.str_('similar'), np.str_('ideas'), np.str_('get'), np.str_('neighbour'), np.str_('thin'), np.str_('latest'), np.str_('partner'), np.str_('appear'), np.str_('meals'), np.str_('suspected'), np.str_('.--"'), np.str_('intelligence'), np.str_('thought'), np.str_('believe'), np.str_('Farm'), np.str_('accidental'), np.str_('staircase'), np.str_('rapidity'), np.str_('contrived'), np.str_('station'), np.str_('had'), np.str_('been'), np.str_('increase'), np.str_('Wallis'), np.str_('conversation'), np.str_('described'), np.str_('another'), np.str_('Papa'), np.str_('compassionate')]


## Congrats! You have completed your assignment. Don't forget to save your notebook (including the cell outputs). Follow the Submission Guideline to complete the submission.